http://www.cs.cmu.edu/~ark/personas/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
%config InlineBackend.figure_format = 'retina'

In [2]:
genres = pd.read_csv(
    'data/raw/movie.metadata.tsv',
    converters={'genres': lambda x: list(eval(x).values())},
    delimiter='\t',
    header=None,
    index_col='id',
    names=['id', 'genres'],
    usecols=[0, 8])

summaries = pd.read_csv(
    'data/raw/plot_summaries.txt',
    delimiter='\t',
    header=None,
    index_col='id',
    names=['id', 'summary'])

df = (
    summaries
    .merge(genres, on='id')
    .reset_index(drop=True))

mlb = MultiLabelBinarizer()
pd.DataFrame(mlb.fit_transform(df.genres), columns=mlb.classes_)

,Absurdism,Acid western,Action,Action Comedy,Action Thrillers,Action/Adventure,Addiction Drama,Adult,Adventure,Adventure Comedy,...,Werewolf fiction,Western,Whodunit,Women in prison films,Workplace Comedy,World History,World cinema,Wuxia,Z movie,Zombie Film
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
